In [ ]:
import numpy as np
from scipy.spatial import Delaunay
from astropy.table import Table as t, vstack
from collections import defaultdict
import os
from astropy.io import fits

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 360
matplotlib.rcParams['text.usetex'] = True
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
from matplotlib.ticker import MaxNLocator, AutoMinorLocator

plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 12,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
})
# plt.style.use('dark_background')
cmap = sns.color_palette('mako', as_cmap=True)
cmap

In [ ]:
def list_links(url):
    resp = urllib.request.urlopen(url)
    soup = BeautifulSoup(resp, 'html.parser')
    return [a['href'] for a in soup.find_all('a', href=True)
            if not a['href'].startswith('?') and a['href'] != '../']

In [ ]:
OUT_DIR = './data/dr1/vac/dr1/lss/guadalupe/v1.0/LSScats/clustering/'

elg = os.listdir(OUT_DIR)
files_elg = [os.path.join(OUT_DIR, fn) for fn in elg]

data = [fn for fn in files_elg if '.dat' in fn]
ran_n = [fn for fn in files_elg if ('_N_' in fn) and ('ran' in fn)]
ran_s = [fn for fn in files_elg if ('_S_' in fn) and ('ran' in fn)]

f_n, f_s = fits.open(data[0]), fits.open(data[1])
data

In [ ]:
table = t.read(f_n, hdu=1)
tiles = np.unique(table['TILES'].data)
print(f'Number of tiles: {len(tiles)}')
table[:5]

In [ ]:
fig, ax = plt.subplots()
ax.hist(table['Z'], bins=20, color=cmap(0.5), edgecolor='black', alpha=0.8,)

ax.xaxis.set_major_locator(MaxNLocator(nbins=10, prune='both'))
ax.yaxis.set_major_locator(MaxNLocator(nbins=10, prune='both'))

ax.xaxis.set_minor_locator(AutoMinorLocator(4))
ax.yaxis.set_minor_locator(AutoMinorLocator(4))

ax.tick_params(which='major', length=6, width=1)
ax.tick_params(which='minor', length=3, width=0.5)

ax.set_xlabel(r'$Z$')
ax.set_ylabel(r'Count')
ax.set_title('ELG redshift distribution')

plt.show()

In [ ]:
tile = table[table['TILES']==tiles[0]]
cuts = (0.91,1.0)

In [ ]:
def stack_randoms(base_dir, n_r=18):
    tables_n = []
    for i in range(n_r):
        tab = t.read(f'{base_dir}ELG_LOPnotqso_N_{i}_clustering.ran.fits', hdu='LSS')
        tables_n.append(tab)
    return vstack(tables_n)

def mask_data(data, rand, tn, cuts):
    mask_d = (data['TILES'] == tn) & (data['Z'] > cuts[0]) & (data['Z'] < cuts[1])
    data_d = data[mask_d]['TARGETID', 'Z', 'RA', 'DEC']

    mask_r = (rand['TILES'] == tn) & (rand['Z'] > cuts[0]) & (rand['Z'] < cuts[1])
    data_r = rand[mask_r]['TARGETID', 'Z', 'RA', 'DEC']

    return data_d, vstack([data_d, data_r])

def get_r(data_d, data_t):
    n_real = len(data_d)
    n_total = len(data_t)
    is_real = np.zeros(n_total, dtype=bool)
    is_real[:n_real] = True

    neighbors = defaultdict(set)
    for simplex in tri.simplices:
        i, j, k = simplex
        neighbors[i].update([j, k])
        neighbors[j].update([i, k])
        neighbors[k].update([i, j])

    connections = {}
    for i in range(n_total):
        nbrs = neighbors[i]
        real_count = sum(is_real[j] for j in nbrs)
        rand_count = len(nbrs) - real_count
        connections[i] = {'real': real_count, 'random': rand_count}
    return connections

In [ ]:
def classify_connections(connections, n_total):
    for i in range(n_total):
        real = connections[i]['real']
        random = connections[i]['random']
        r = (real-random)/(real+random)
        if -1.0 <= r and r <= -0.9:
            type = 'void'
        elif -0.9 < r and r <= 0.0:
            type = 'sheet'
        elif 0.0 < r and r <= 0.9:
            type = 'filament'
        elif 0.9 < r and r <= 1.0:
            type = 'knot'
        else:
            break
            print(f'Error: r={r}')

        connections[i]['r'] = r
        connections[i]['type'] = type
    return connections

def classify_data(data_t, tri_data, connections):
    voids, sheets, filaments, knots = [], [], [], []

    for key, value in connections.items():
        data_t = tri_data[key]
        if value['type'] == 'void':
            voids.append(data_t)
        elif value['type'] == 'sheet':
            sheets.append(data_t)
        elif value['type'] == 'filament':
            filaments.append(data_t)
        elif value['type'] == 'knot':
            knots.append(data_t)
        else:
            print(f'Error: {value["type"]}')
    return voids, sheets, filaments, knots

In [ ]:
rand = stack_randoms(OUT_DIR)
tile_id = tiles[0]

data_d, data_t = mask_data(table, rand, tile_id, cuts)
tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
tri = Delaunay(tri_data)

connections = get_r(data_d, data_t)
connections_ = classify_connections(connections, len(data_t))

voids, sheets, filaments, knots = classify_data(data_t, tri_data, connections_)
print(f'Tile {tile_id}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

mask = (tile['Z'] > cuts[0]) & (tile['Z'] < cuts[1])

data_r = tile[mask]
ra, dec = data_r['RA'], data_r['DEC']

axes[0].grid(linewidth=0.2, color='gray')
axes[0].scatter(tile['RA'], tile['DEC'], color=cmap(0.5), alpha=0.8, label='Data')
axes[0].set_xlabel('RA [deg]')
axes[0].set_box_aspect(1)
axes[0].legend()

axes[1].grid(linewidth=0.2, color='gray')
axes[1].scatter(ra, dec, color=cmap(0.7), alpha=0.8, label='Filtered Data')
axes[1].set_xlabel('RA [deg]')
axes[1].set_box_aspect(1)
axes[1].legend()

plt.suptitle(f'Tile {tile_id}\n'+f'{cuts[0]} '+r'$< Z <$'+f' {cuts[1]}')
plt.show()

In [ ]:
rand

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

mask = (tile['Z'] > cuts[0]) & (tile['Z'] < cuts[1])
mask_r = (rand['TILES']==tile_id) &  (rand['Z'] > cuts[0]) & (rand['Z'] < cuts[1])

data_r = tile[mask]
ra, dec = data_r['RA'], data_r['DEC']

axes[0].grid(linewidth=0.2, color='gray')
axes[0].scatter(tile['RA'], tile['DEC'], color=cmap(0.5), alpha=0.8, label='Data')
axes[0].scatter(rand['RA'], rand['DEC'], color=cmap(0.7), alpha=0.5, label='Random')
axes[0].set_xlabel('RA [deg]')
axes[0].set_box_aspect(1)
axes[0].legend()

axes[1].grid(linewidth=0.2, color='gray')
axes[1].scatter(ra, dec, color=cmap(0.7), alpha=0.8, label='Random')
axes[1].scatter(rand[mask_r]['RA'], rand[mask_r]['DEC'], color=cmap(0.5), alpha=0.5, label='Filtered Random')
axes[1].set_xlabel('RA [deg]')
axes[1].set_box_aspect(1)
axes[1].legend()

plt.suptitle(f'Tile {tile_id}\n'+f'{cuts[0]} '+r'$< Z <$'+f' {cuts[1]}')
plt.show()